### This notebook is for experimenting with the Keras Conv1D layer to see if a moving window convolution could be used.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, MaxPooling1D, Conv2D, Conv1D, InputLayer, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.initializers import Initializer, glorot_uniform
from keras.constraints import Constraint
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

C:\Users\stone\Miniconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def create_conv_flux_model():
    model = Sequential()
    model.add(InputLayer(batch_input_shape=(None,4639, 1)))
    model.add(Conv1D(filters=64, kernel_size=2,activation= 'relu'))
    model.add(Conv1D(filters=24, kernel_size=2,activation= 'relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(30,activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(20, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [4]:
fake_data = np.ones((1,100,1))

In [2]:
class WindowedInitalizer(Initializer):
    """
    Initalizer used for the Windowed Convolution.
    """    

    def __init__(self, window_vect):
        """
        Saves window vect given.
        """
        self.window_vect = window_vect
    
    def __call__(self, shape, dtype=None):
        """
        Method to call when generating initil kernel.
        """
        
        # Calculates the shape for one row of the kernel.
        window_shape = (1, shape[1], shape[2])

        # Generate the kernel based on the window vector..
        weight_list = []
        for val in self.window_vect:
            weight_list.append(glorot_uniform()(window_shape, dtype) if val == 1 else K.constant(0, shape=window_shape))

        # Join all the rows together and make sure the shape checks out.
        weights = K.concatenate(weight_list, 0)
        if weights.shape != shape:
            raise KeyError('WRONG')
            
        return weights

class WindowedConstraint(Constraint):
    """
    Constraint to apply along side with the windowed convolution.
    """
    
    
    def __init__(self, window_vect):
        """
        Make the matrix used to constraint the kernel.
        """
        self.window_vect = window_vect
        self.constraint_matrix = tf.cast(tf.diag(self.window_vect), 'float32')
    
    def __call__(self, w):
        """
        Apply the constraint to the kernel.
        """
        w = K.dot(K.transpose(w), self.constraint_matrix)
        w = K.transpose(w)
        return w

# Example Usage.    
window_vector = [1,1,1,0,0,0,0,1,1,1]
second_window_vector = [1,0,0,0,1]
def create_model():
    model = Sequential()
    model.add(InputLayer(batch_input_shape=(None, 100, 2)))
    model.add(Conv1D(filters=5, kernel_size=10,
                     kernel_initializer=WindowedInitalizer(window_vector), kernel_constraint=WindowedConstraint(window_vector), use_bias=False))
    model.add(Conv1D(filters=2, kernel_size=5, 
                    kernel_initializer=WindowedInitalizer(second_window_vector), kernel_constraint=WindowedConstraint(second_window_vector), use_bias=False))
    model.compile('adam', 'binary_crossentropy')
    return  model
mod = create_model()

In [31]:
x = tf.convert_to_tensor(mod.get_weights()[1])
tf.Session().run(x)

array([[[ 0.98666847]],

       [[ 0.        ]],

       [[ 0.        ]],

       [[ 0.        ]],

       [[-0.24127936]]], dtype=float32)

In [110]:
mod.predict(fake_data[:1]).shape

(1, 87, 1)

In [4]:
weights = mod.get_weights()[0]
weights, weights.shape

(array([[[ 0.33609426,  0.54662156, -0.697273  , -0.5697129 ,
          -0.15130985],
         [-0.08651352, -0.5406983 ,  0.45375144, -0.62430406,
          -0.47682396]],
 
        [[ 0.03792542, -0.6094558 , -0.90444213,  0.1141597 ,
           0.477414  ],
         [-0.03760666,  0.10133624,  0.25666845, -0.7138198 ,
          -0.28372544]],
 
        [[ 0.05919898,  0.67914176,  0.8845098 ,  0.42968512,
          -0.790707  ],
         [-0.20054871,  0.77023244,  0.14707506, -0.51781666,
          -0.7059072 ]],
 
        [[ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ]],
 
        [[ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ]],
 
        [[ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.

In [3]:
weights = mod.get_weights()[1]
weights, weights.shape

(array([[[ 0.14928663, -0.46126118],
         [-0.7388681 ,  0.81229126],
         [-0.14351666, -0.01900131],
         [-0.8659764 , -0.23624492],
         [ 0.31248045,  0.7684511 ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.7080611 , -0.56604826],
         [ 0.8290849 , -0.53153384],
         [-0.34839898, -0.16457921],
         [ 0.04644638,  0.14801502],
         [ 0.6786972 , -0.61518496]]], dtype=float32), (5, 5, 2))

In [8]:
fake_labels = np.random.randint(0,100, (10,87,2))
fake_data = np.random.randint(0,100, (10,100,2))

In [12]:
mod.fit(fake_data, fake_labels, epochs=10)

Epoch 1/10
10/10 [==============================] - 0s 757us/step - loss: -217.9510
Epoch 2/10
10/10 [==============================] - 0s 652us/step - loss: -222.4864
Epoch 3/10
10/10 [==============================] - 0s 602us/step - loss: -226.9231
Epoch 4/10
10/10 [==============================] - 0s 652us/step - loss: -232.7534
Epoch 5/10
10/10 [==============================] - 0s 599us/step - loss: -236.7568
Epoch 6/10
10/10 [==============================] - 0s 501us/step - loss: -239.3979
Epoch 7/10
10/10 [==============================] - 0s 599us/step - loss: -244.0278
Epoch 8/10
10/10 [==============================] - 0s 599us/step - loss: -247.8200
Epoch 9/10
10/10 [==============================] - 0s 604us/step - loss: -250.9548
Epoch 10/10
10/10 [==============================] - 0s 605us/step - loss: -254.2245


In [13]:
mod.get_weights()[1]

array([[[ 0.1588422 , -0.4526687 ],
        [-0.7247966 ,  0.82106525],
        [-0.14960739, -0.02355396],
        [-0.87910753, -0.24500136],
        [ 0.3020743 ,  0.75946367]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.7181156 , -0.55724394],
        [ 0.8424545 , -0.52256763],
        [-0.35970205, -0.17073224],
        [ 0.033482  ,  0.13889179],
        [ 0.66780937, -0.6241443 ]]], dtype=float32)

In [11]:
mod.get_weights()[0]

array([[[ 0.3299878 ,  0.5493097 , -0.70447546, -0.5776629 ,
         -0.14647587],
        [-0.09282469, -0.532383  ,  0.44565874, -0.6329118 ,
         -0.46787515]],

       [[ 0.03192651, -0.6066033 , -0.91174626,  0.11035498,
          0.48055667],
        [-0.0442126 ,  0.10730062,  0.24959145, -0.7208096 ,
         -0.2765992 ]],

       [[ 0.05271503,  0.6802243 ,  0.87725013,  0.4220895 ,
         -0.78313047],
        [-0.20686646,  0.7703582 ,  0.13970111, -0.52505445,
         -0.6988258 ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
     